In [ ]:
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from itertools import chain

# Load the datasets
train_file_path = '/content/Tam-SA-train.csv'
test_file_path = '/content/Tam-SA-test-without-labels.csv'

train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# Analyze class distribution
print(train_data['Label'].value_counts(normalize=True))

# Preprocessing function for text data
def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()    # Remove extra spaces
    return text.lower()

# Apply preprocessing
train_data['Cleaned_Text'] = train_data['Text'].apply(preprocess_text)
test_data['Cleaned_Text'] = test_data['Text'].apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
train_data['Encoded_Label'] = label_encoder.fit_transform(train_data['Label'])

# Define label mapping for reference
label_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print("Label Mapping:", label_mapping)

# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train_data['Cleaned_Text'], train_data['Encoded_Label'],
    test_size=0.2, random_state=42, stratify=train_data['Encoded_Label']
)

# Build vocabulary
def build_vocab(sentences, vocab_size=20000):
    words = chain(*[sentence.split() for sentence in sentences])
    most_common = Counter(words).most_common(vocab_size - 2)
    word_to_idx = {word: idx + 2 for idx, (word, _) in enumerate(most_common)}
    word_to_idx["<PAD>"] = 0
    word_to_idx["<UNK>"] = 1
    return word_to_idx

vocab = build_vocab(X_train, vocab_size=20000)
vocab_size = len(vocab)

# Custom dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_length=100):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.vocab = vocab
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def encode_text(self, text):
        tokens = text.split()
        encoded = [self.vocab.get(token, self.vocab["<UNK>"]) for token in tokens]
        padded = encoded[:self.max_length] + [self.vocab["<PAD>"]] * max(0, self.max_length - len(encoded))
        return padded

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return torch.tensor(self.encode_text(text), dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Create datasets and data loaders
train_dataset = TextDataset(X_train, y_train, vocab, max_length=100)
val_dataset = TextDataset(X_val, y_val, vocab, max_length=100)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Define CNN model
class CNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, num_classes, kernel_sizes, num_channels):
        super(CNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_size, out_channels=num_channels, kernel_size=ks)
            for ks in kernel_sizes
        ])
        self.fc = nn.Linear(num_channels * len(kernel_sizes), num_classes)

    def forward(self, x):
        embedded = self.embedding(x).permute(0, 2, 1)
        conv_outs = [torch.relu(conv(embedded)).max(dim=2)[0] for conv in self.convs]
        out = torch.cat(conv_outs, dim=1)
        return self.fc(out)

# Initialize model, loss, and optimizer
embed_size = 128
num_classes = len(label_mapping)
kernel_sizes = [3, 4, 5]
num_channels = 100

model = CNNModel(vocab_size, embed_size, num_classes, kernel_sizes, num_channels)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=5):
    for epoch in range(epochs):
        model.train()
        train_loss, train_correct = 0, 0
        for texts, labels in train_loader:
            texts, labels = texts.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_correct += (outputs.argmax(1) == labels).sum().item()

        val_loss, val_correct = 0, 0
        model.eval()
        with torch.no_grad():
            for texts, labels in val_loader:
                texts, labels = texts.to(device), labels.to(device)
                outputs = model(texts)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += (outputs.argmax(1) == labels).sum().item()

        print(f"Epoch {epoch + 1}/{epochs}: "
              f"Train Loss: {train_loss / len(train_loader):.4f}, "
              f"Train Acc: {train_correct / len(train_loader.dataset):.4f}, "
              f"Val Loss: {val_loss / len(val_loader):.4f}, "
              f"Val Acc: {val_correct / len(val_loader.dataset):.4f}")

train_model(model, criterion, optimizer, train_loader, val_loader, epochs=5)

# Predict function for the test dataset
def predict_test(model, test_texts, vocab, max_length=100):
    model.eval()
    encoded_texts = [
        torch.tensor([vocab.get(token, vocab["<UNK>"]) for token in text.split()][:max_length] +
                     [vocab["<PAD>"]] * max(0, max_length - len(text.split())),
                     dtype=torch.long)
        for text in test_texts
    ]
    test_dataset = torch.stack(encoded_texts)
    test_loader = DataLoader(test_dataset, batch_size=32)

    predictions = []
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            outputs = model(batch)
            predictions.extend(outputs.argmax(1).cpu().numpy())
    return predictions

# Get predictions for the test file
test_texts = test_data['Cleaned_Text']
predicted_labels = predict_test(model, test_texts, vocab, max_length=100)

# Map predicted numeric labels back to original labels
predicted_labels_decoded = label_encoder.inverse_transform(predicted_labels)

# Save predictions to CSV
output_df = pd.DataFrame({
    "Text": test_data['Text'],
    "Predicted_Label": predicted_labels_decoded
})
output_csv_path = '/content/drive/MyDrive/codalab/tam_CNN_prediction.csv'
output_df.to_csv(output_csv_path, index=False)
print(f"Predictions saved to {output_csv_path}")


Label
Positive          0.583028
unknown_state     0.165928
Negative          0.133378
Mixed_feelings    0.117666
Name: proportion, dtype: float64
Label Mapping: {0: 'Mixed_feelings', 1: 'Negative', 2: 'Positive', 3: 'unknown_state'}
Epoch 1/5: Train Loss: 1.0549, Train Acc: 0.5916, Val Loss: 1.0578, Val Acc: 0.5667
Epoch 2/5: Train Loss: 0.8716, Train Acc: 0.6676, Val Loss: 1.0655, Val Acc: 0.5986
Epoch 3/5: Train Loss: 0.6130, Train Acc: 0.7805, Val Loss: 1.2015, Val Acc: 0.5969
Epoch 4/5: Train Loss: 0.3807, Train Acc: 0.8727, Val Loss: 1.3900, Val Acc: 0.5740
Epoch 5/5: Train Loss: 0.2776, Train Acc: 0.9082, Val Loss: 1.5849, Val Acc: 0.5841
Predictions saved to /content/drive/MyDrive/codalab/tam_CNN_prediction.csv


In [3]:
# Define and load model (if needed)
model = CNNModel(vocab_size, embed_size, num_classes, kernel_sizes, num_channels)
model.load_state_dict(torch.load("model_checkpoint.pth"))  # Load checkpoint if saved
model.to(device)
model.eval()


NameError: name 'CNNModel' is not defined

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Get model predictions on the validation set
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for texts, labels in val_loader:
        texts, labels = texts.to(device), labels.to(device)
        outputs = model(texts)
        preds = outputs.argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)
labels = [label_mapping[i] for i in range(len(label_mapping))]  # Get class labels

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()


NameError: name 'model' is not defined

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import (
    XLNetTokenizer,
    XLNetForSequenceClassification,
    Trainer,
    TrainingArguments,
)

# Load the datasets
train_file_path = '/content/Tam-SA-train.csv'
test_file_path = '/content/Tam-SA-test-without-labels.csv'

train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# Preprocess the text data
def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()    # Remove extra spaces
    return text.lower()

train_data['Cleaned_Text'] = train_data['Text'].apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
train_data['Encoded_Label'] = label_encoder.fit_transform(train_data['Label'])

# Create label mapping
label_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print("Label Mapping:", label_mapping)

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train_data['Cleaned_Text'], train_data['Encoded_Label'],
    test_size=0.2, random_state=42, stratify=train_data['Encoded_Label']
)

# Define a custom dataset for tokenized text
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

# Initialize tokenizer and datasets
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
max_length = 64  # Reduced max length for faster training

train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length)
val_dataset = SentimentDataset(X_val, y_val, tokenizer, max_length)

# Load XLNet model
num_labels = len(label_mapping)
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=num_labels)

# Freeze lower layers
for param in model.transformer.layer[:6]:
    param.requires_grad = False

# Define training arguments with fewer epochs and mixed precision
training_args = TrainingArguments(
    output_dir="./xlnet_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Smaller batch size for speed
    per_device_eval_batch_size=8,
    num_train_epochs=2,  # Reduced epochs
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

# Define a compute_metrics function for evaluation
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Save the trained model and tokenizer
model.save_pretrained("./xlnet_model")
tokenizer.save_pretrained("./xlnet_model")

# Evaluate the model on the validation dataset
eval_results = trainer.evaluate()
print(f"Validation Accuracy: {eval_results['eval_accuracy']}")


Label Mapping: {0: 'Mixed_feelings', 1: 'Negative', 2: 'Positive', 3: 'unknown_state'}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


In [ ]:
# Preprocess and tokenize the test data
test_data['Cleaned_Text'] = test_data['Text'].apply(preprocess_text)

class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts.tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
        }

# Create test dataset and loader
test_dataset = TestDataset(test_data['Cleaned_Text'], tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=8)

# Make predictions on the test dataset
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(training_args.device)
        attention_mask = batch['attention_mask'].to(training_args.device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(preds.cpu().tolist())

# Map predictions to labels
test_data['Predicted_Label'] = [label_mapping[pred] for pred in predictions]

# Save the predictions to a CSV file
output_file_path = '/content/drive/MyDrive/codalab/tam_XLNET_prediction.csv'
test_data[['Text', 'Predicted_Label']].to_csv(output_file_path, index=False)
print(f"Predictions saved to {output_file_path}")
